# 启用人工干预

In [ ]:
"""
LangGraph人工干预循环演示代码集合
基于旅游规划系统的完整人工干预功能演示

演示包括：
1. 基础人工干预演示
2. 批准/拒绝模式演示  
3. 状态编辑演示
4. 工具调用审查演示
5. 输入验证演示
6. 多重中断演示
7. 完整旅游规划人工干预工作流
"""


In [1]:

import uuid
import time
import json
from typing import TypedDict, Annotated, List, Dict, Any, Optional, Literal
from typing_extensions import TypedDict

from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# 导入现有的智能体和工具
from supervisor_agent import (
    llm, tour_search_agent, day_plan_agent, 
    live_transport_agent, travel_butler_agent,
    get_geocodes, search_nearby, tour_search
)

# ==================== 演示1: 基础人工干预演示 ====================

class BasicInterruptState(TypedDict):
    """基础中断状态"""
    user_input: str
    search_results: str
    confirmed_results: str
    status: List[str]

def basic_tour_search_node(state: BasicInterruptState) -> BasicInterruptState:
    """景点搜索节点 - 带人工确认"""
    print("🔍 开始景点搜索...")
    
    # 调用景点搜索智能体
    result = tour_search_agent.invoke({
        "messages": [{"role": "user", "content": state["user_input"]}]
    })
    
    search_results = result["messages"][-1].content
    
    # 人工干预点：让用户确认搜索结果
    user_feedback = interrupt({
        "task": "请确认景点搜索结果",
        "search_results": search_results,
        "question": "这些景点信息是否符合您的需求？如需修改，请提供具体建议。"
    })
    
    return {
        "search_results": search_results,
        "confirmed_results": user_feedback,
        "status": ["search_completed_with_confirmation"]
    }

def create_basic_interrupt_demo():
    """创建基础人工干预演示工作流"""
    workflow = StateGraph(BasicInterruptState)
    
    workflow.add_node("search_with_interrupt", basic_tour_search_node)
    workflow.set_entry_point("search_with_interrupt")
    workflow.add_edge("search_with_interrupt", END)
    
    return workflow.compile(checkpointer=MemorySaver())


In [2]:
workflow = create_basic_interrupt_demo()

In [ ]:

# ==================== 演示2: 批准/拒绝模式演示 ====================

class ApprovalState(TypedDict):
    """批准/拒绝状态"""
    user_input: str
    day_plan: str
    approval_status: str
    final_plan: str
    status: List[str]

def day_plan_approval_node(state: ApprovalState) -> Command[Literal["approved_path", "rejected_path"]]:
    """行程规划批准节点"""
    print("📅 生成行程规划...")
    
    # 生成行程规划
    result = day_plan_agent.invoke({
        "messages": [{"role": "user", "content": state["user_input"]}]
    })
    
    day_plan = result["messages"][-1].content
    
    # 人工干预：批准或拒绝
    approval = interrupt({
        "question": "请审查以下行程规划，是否批准？",
        "day_plan": day_plan,
        "options": ["approve", "reject"],
        "instructions": "回复 'approve' 批准，'reject' 拒绝并重新规划"
    })
    
    if approval == "approve":
        return Command(goto="approved_path", update={"day_plan": day_plan, "approval_status": "approved"})
    else:
        return Command(goto="rejected_path", update={"day_plan": day_plan, "approval_status": "rejected"})

def approved_plan_node(state: ApprovalState) -> ApprovalState:
    """批准路径节点"""
    print("✅ 行程已批准，继续后续规划...")
    return {
        "final_plan": f"【已批准】{state['day_plan']}",
        "status": ["plan_approved"]
    }

def rejected_plan_node(state: ApprovalState) -> ApprovalState:
    """拒绝路径节点"""
    print("❌ 行程被拒绝，需要重新规划...")
    
    # 这里可以添加重新规划的逻辑
    new_plan = interrupt({
        "question": "请提供重新规划的要求",
        "rejected_plan": state["day_plan"],
        "instructions": "请说明需要如何调整行程"
    })
    
    return {
        "final_plan": f"【需重新规划】基于反馈：{new_plan}",
        "status": ["plan_rejected_need_revision"]
    }

def create_approval_demo():
    """创建批准/拒绝演示工作流"""
    workflow = StateGraph(ApprovalState)
    
    workflow.add_node("day_plan_approval", day_plan_approval_node)
    workflow.add_node("approved_path", approved_plan_node)
    workflow.add_node("rejected_path", rejected_plan_node)
    
    workflow.set_entry_point("day_plan_approval")
    workflow.add_edge("approved_path", END)
    workflow.add_edge("rejected_path", END)
    
    return workflow.compile(checkpointer=MemorySaver())

# ==================== 演示3: 状态编辑演示 ====================

class EditableState(TypedDict):
    """可编辑状态"""
    user_input: str
    budget: int
    transport_preference: str
    transport_plan: str
    edited_plan: str
    status: List[str]

def transport_planning_node(state: EditableState) -> EditableState:
    """交通规划节点 - 支持状态编辑"""
    print("🚗 开始交通规划...")
    
    # 首先让用户确认/编辑预算和偏好
    user_preferences = interrupt({
        "task": "请确认或编辑您的交通偏好",
        "current_budget": state.get("budget", 1000),
        "current_preference": state.get("transport_preference", "经济实惠"),
        "editable_fields": {
            "budget": "预算金额（元）",
            "transport_preference": "交通偏好（经济实惠/舒适便捷/时间优先）"
        },
        "instructions": "请提供JSON格式：{'budget': 数字, 'transport_preference': '偏好'}"
    })
    
    # 解析用户编辑的偏好
    try:
        if isinstance(user_preferences, str):
            import json
            preferences = json.loads(user_preferences)
        else:
            preferences = user_preferences
            
        updated_budget = preferences.get("budget", state.get("budget", 1000))
        updated_preference = preferences.get("transport_preference", state.get("transport_preference", "经济实惠"))
    except:
        # 如果解析失败，使用原值
        updated_budget = state.get("budget", 1000)
        updated_preference = state.get("transport_preference", "经济实惠")
    
    # 基于更新的偏好生成交通规划
    planning_input = f"""
    用户需求：{state['user_input']}
    预算：{updated_budget}元
    偏好：{updated_preference}
    请根据这些信息制定交通规划。
    """
    
    result = live_transport_agent.invoke({
        "messages": [{"role": "user", "content": planning_input}]
    })
    
    return {
        "budget": updated_budget,
        "transport_preference": updated_preference,
        "transport_plan": result["messages"][-1].content,
        "status": ["transport_planned_with_edits"]
    }

def create_editable_state_demo():
    """创建状态编辑演示工作流"""
    workflow = StateGraph(EditableState)
    
    workflow.add_node("transport_planning", transport_planning_node)
    workflow.set_entry_point("transport_planning")
    workflow.add_edge("transport_planning", END)
    
    return workflow.compile(checkpointer=MemorySaver())

# ==================== 演示4: 工具调用审查演示 ====================

class ToolReviewState(TypedDict):
    """工具调用审查状态"""
    user_input: str
    tool_calls: List[Dict]
    approved_calls: List[Dict]
    results: str
    status: List[str]

def reviewed_geocode_tool(addresses: List[str]) -> str:
    """带审查的地理编码工具"""
    # 人工干预：审查工具调用
    approval = interrupt({
        "tool_name": "get_geocodes",
        "parameters": {"addresses": addresses},
        "question": "即将调用地理编码API查询以下地址的坐标，是否继续？",
        "addresses": addresses,
        "estimated_cost": f"预计消耗{len(addresses)}次API调用",
        "options": ["approve", "edit", "cancel"]
    })
    
    if approval["action"] == "approve":
        # 调用原始工具
        return get_geocodes(addresses)
    elif approval["action"] == "edit":
        # 使用编辑后的参数
        edited_addresses = approval.get("edited_addresses", addresses)
        return get_geocodes(edited_addresses)
    else:
        return "❌ 工具调用已取消"

def geocode_review_node(state: ToolReviewState) -> ToolReviewState:
    """地理编码审查节点"""
    print("📍 准备查询地理坐标...")
    
    # 从用户输入中提取地址（简化示例）
    addresses = ["北京天安门", "北京故宫", "北京颐和园"]
    
    # 调用带审查的工具
    result = reviewed_geocode_tool(addresses)
    
    return {
        "tool_calls": [{"tool": "get_geocodes", "params": addresses}],
        "results": result,
        "status": ["geocoding_completed_with_review"]
    }

def create_tool_review_demo():
    """创建工具调用审查演示工作流"""
    workflow = StateGraph(ToolReviewState)
    
    workflow.add_node("geocode_review", geocode_review_node)
    workflow.set_entry_point("geocode_review")
    workflow.add_edge("geocode_review", END)
    
    return workflow.compile(checkpointer=MemorySaver())

# ==================== 演示5: 输入验证演示 ====================

class ValidationState(TypedDict):
    """输入验证状态"""
    user_input: str
    travel_days: int
    budget: int
    validated_input: Dict[str, Any]
    status: List[str]

def input_validation_node(state: ValidationState) -> ValidationState:
    """输入验证节点"""
    print("✅ 开始验证用户输入...")
    
    validated_data = {}
    
    # 验证旅行天数
    while True:
        days_input = interrupt({
            "question": "请输入您的旅行天数（1-30天）",
            "current_input": state.get("travel_days", ""),
            "validation_rule": "必须是1-30之间的整数"
        })
        
        try:
            days = int(days_input)
            if 1 <= days <= 30:
                validated_data["travel_days"] = days
                break
            else:
                continue  # 重新询问
        except ValueError:
            continue  # 重新询问
    
    # 验证预算
    while True:
        budget_input = interrupt({
            "question": "请输入您的预算（500-50000元）",
            "current_input": state.get("budget", ""),
            "validation_rule": "必须是500-50000之间的整数"
        })
        
        try:
            budget = int(budget_input)
            if 500 <= budget <= 50000:
                validated_data["budget"] = budget
                break
            else:
                continue  # 重新询问
        except ValueError:
            continue  # 重新询问
    
    return {
        "validated_input": validated_data,
        "status": ["input_validated"]
    }

def create_validation_demo():
    """创建输入验证演示工作流"""
    workflow = StateGraph(ValidationState)
    
    workflow.add_node("input_validation", input_validation_node)
    workflow.set_entry_point("input_validation")
    workflow.add_edge("input_validation", END)
    
    return workflow.compile(checkpointer=MemorySaver())

# ==================== 演示6: 多重中断演示 ====================

class MultiInterruptState(TypedDict):
    """多重中断状态"""
    user_input: str
    search_confirmed: bool
    plan_approved: bool
    transport_confirmed: bool
    all_confirmations: Dict[str, Any]
    status: List[str]

def multi_interrupt_node(state: MultiInterruptState) -> MultiInterruptState:
    """多重中断节点"""
    print("🔄 开始多重确认流程...")
    
    # 第一个中断：景点确认
    search_confirmation = interrupt({
        "step": 1,
        "task": "景点搜索确认",
        "question": "请确认景点选择",
        "interrupt_id": "search_confirm"
    })
    
    # 第二个中断：行程确认
    plan_confirmation = interrupt({
        "step": 2,
        "task": "行程安排确认", 
        "question": "请确认行程安排",
        "interrupt_id": "plan_confirm"
    })
    
    # 第三个中断：交通确认
    transport_confirmation = interrupt({
        "step": 3,
        "task": "交通方案确认",
        "question": "请确认交通方案",
        "interrupt_id": "transport_confirm"
    })
    
    return {
        "search_confirmed": search_confirmation,
        "plan_approved": plan_confirmation,
        "transport_confirmed": transport_confirmation,
        "all_confirmations": {
            "search": search_confirmation,
            "plan": plan_confirmation,
            "transport": transport_confirmation
        },
        "status": ["multi_interrupt_completed"]
    }

def create_multi_interrupt_demo():
    """创建多重中断演示工作流"""
    workflow = StateGraph(MultiInterruptState)
    
    workflow.add_node("multi_interrupt", multi_interrupt_node)
    workflow.set_entry_point("multi_interrupt")
    workflow.add_edge("multi_interrupt", END)
    
    return workflow.compile(checkpointer=MemorySaver())

# ==================== 演示7: 完整旅游规划人工干预工作流 ====================

class CompleteTravelState(TypedDict):
    """完整旅游规划状态"""
    user_input: str
    collected_needs: Dict[str, Any]
    search_results: str
    day_plan: str
    transport_plan: str
    butler_suggestions: str
    final_travel_guide: str
    user_confirmations: Dict[str, Any]
    status: List[str]

def interactive_need_collection_node(state: CompleteTravelState) -> CompleteTravelState:
    """交互式需求收集节点"""
    print("📋 开始需求收集...")
    
    # 基础需求收集
    basic_needs = interrupt({
        "task": "旅游需求收集",
        "question": "请提供您的基本旅游需求",
        "required_fields": {
            "destination": "目的地",
            "duration": "旅行天数",
            "travelers": "出行人数和类型",
            "budget": "预算范围"
        },
        "instructions": "请提供JSON格式的需求信息"
    })
    
    return {
        "collected_needs": basic_needs,
        "status": ["needs_collected"]
    }

def interactive_search_node(state: CompleteTravelState) -> CompleteTravelState:
    """交互式景点搜索节点"""
    print("🔍 开始景点搜索...")
    
    # 基于需求搜索景点
    search_input = f"根据用户需求搜索景点：{state['collected_needs']}"
    result = tour_search_agent.invoke({
        "messages": [{"role": "user", "content": search_input}]
    })
    
    search_results = result["messages"][-1].content
    
    # 用户确认搜索结果
    user_feedback = interrupt({
        "task": "景点搜索结果确认",
        "search_results": search_results,
        "question": "这些景点是否符合您的需求？需要调整吗？",
        "options": ["确认", "需要调整", "重新搜索"]
    })
    
    # 根据反馈处理
    if user_feedback == "需要调整":
        adjustment = interrupt({
            "question": "请说明需要如何调整景点选择",
            "current_results": search_results
        })
        final_results = f"{search_results}\n\n【用户调整要求】{adjustment}"
    else:
        final_results = search_results
    
    return {
        "search_results": final_results,
        "user_confirmations": {"search_confirmed": user_feedback},
        "status": ["search_completed"]
    }

def interactive_planning_node(state: CompleteTravelState) -> CompleteTravelState:
    """交互式行程规划节点"""
    print("📅 开始行程规划...")
    
    # 生成行程规划
    planning_input = f"""
    用户需求：{state['collected_needs']}
    景点信息：{state['search_results']}
    请制定详细的行程规划。
    """
    
    result = day_plan_agent.invoke({
        "messages": [{"role": "user", "content": planning_input}]
    })
    
    day_plan = result["messages"][-1].content
    
    # 用户审查行程
    plan_approval = interrupt({
        "task": "行程规划审查",
        "day_plan": day_plan,
        "question": "请审查行程安排，是否需要调整？",
        "options": ["批准", "需要调整", "重新规划"]
    })
    
    if plan_approval == "需要调整":
        adjustments = interrupt({
            "question": "请说明需要如何调整行程",
            "current_plan": day_plan
        })
        final_plan = f"{day_plan}\n\n【调整要求】{adjustments}"
    else:
        final_plan = day_plan
    
    return {
        "day_plan": final_plan,
        "user_confirmations": {
            **state.get("user_confirmations", {}),
            "plan_approved": plan_approval
        },
        "status": ["planning_completed"]
    }

def interactive_transport_node(state: CompleteTravelState) -> CompleteTravelState:
    """交互式交通规划节点"""
    print("🚗 开始交通规划...")
    
    # 生成交通规划
    transport_input = f"""
    用户需求：{state['collected_needs']}
    行程安排：{state['day_plan']}
    请制定交通和住宿方案。
    """
    
    result = live_transport_agent.invoke({
        "messages": [{"role": "user", "content": transport_input}]
    })
    
    transport_plan = result["messages"][-1].content
    
    # 用户确认交通方案
    transport_approval = interrupt({
        "task": "交通方案确认",
        "transport_plan": transport_plan,
        "question": "请确认交通和住宿安排",
        "options": ["确认", "调整预算", "更换交通方式"]
    })
    
    if transport_approval != "确认":
        modifications = interrupt({
            "question": f"您选择了'{transport_approval}'，请提供具体要求",
            "current_plan": transport_plan
        })
        final_transport = f"{transport_plan}\n\n【修改要求】{modifications}"
    else:
        final_transport = transport_plan
    
    return {
        "transport_plan": final_transport,
        "user_confirmations": {
            **state.get("user_confirmations", {}),
            "transport_confirmed": transport_approval
        },
        "status": ["transport_completed"]
    }

def interactive_butler_node(state: CompleteTravelState) -> CompleteTravelState:
    """交互式旅行管家节点"""
    print("💡 开始旅行建议...")
    
    # 生成旅行建议
    butler_input = f"""
    用户需求：{state['collected_needs']}
    行程安排：{state['day_plan']}
    交通住宿：{state['transport_plan']}
    请提供旅行建议和注意事项。
    """
    
    result = travel_butler_agent.invoke({
        "messages": [{"role": "user", "content": butler_input}]
    })
    
    butler_suggestions = result["messages"][-1].content
    
    # 用户确认建议
    suggestions_feedback = interrupt({
        "task": "旅行建议确认",
        "suggestions": butler_suggestions,
        "question": "这些建议是否有帮助？还需要其他信息吗？",
        "options": ["很有帮助", "需要补充", "不太相关"]
    })
    
    if suggestions_feedback == "需要补充":
        additional_requests = interrupt({
            "question": "请说明还需要什么额外信息或建议",
            "current_suggestions": butler_suggestions
        })
        final_suggestions = f"{butler_suggestions}\n\n【补充信息】{additional_requests}"
    else:
        final_suggestions = butler_suggestions
    
    return {
        "butler_suggestions": final_suggestions,
        "user_confirmations": {
            **state.get("user_confirmations", {}),
            "suggestions_confirmed": suggestions_feedback
        },
        "status": ["butler_completed"]
    }

def final_integration_node(state: CompleteTravelState) -> CompleteTravelState:
    """最终整合节点"""
    print("📝 生成最终旅游攻略...")
    
    # 整合所有信息
    final_guide = f"""
# 🌟 个性化旅游攻略

## 📋 需求确认
{state['collected_needs']}

## 🏞️ 精选景点
{state['search_results']}

## 📅 详细行程
{state['day_plan']}

## 🚗 交通住宿
{state['transport_plan']}

## 💡 贴心建议
{state['butler_suggestions']}

## ✅ 确认记录
{state.get('user_confirmations', {})}

---
*攻略生成时间：{time.strftime('%Y-%m-%d %H:%M:%S')}*
*本攻略基于您的确认和反馈定制生成*
    """
    
    # 最终确认
    final_approval = interrupt({
        "task": "最终攻略确认",
        "final_guide": final_guide,
        "question": "请确认最终的旅游攻略是否满意",
        "options": ["满意", "需要微调", "重新制定"]
    })
    
    return {
        "final_travel_guide": final_guide,
        "user_confirmations": {
            **state.get("user_confirmations", {}),
            "final_approved": final_approval
        },
        "status": ["completed"]
    }

def create_complete_interactive_workflow():
    """创建完整的交互式旅游规划工作流"""
    workflow = StateGraph(CompleteTravelState)
    
    # 添加所有节点
    workflow.add_node("need_collection", interactive_need_collection_node)
    workflow.add_node("search_interaction", interactive_search_node)
    workflow.add_node("planning_interaction", interactive_planning_node)
    workflow.add_node("transport_interaction", interactive_transport_node)
    workflow.add_node("butler_interaction", interactive_butler_node)
    workflow.add_node("final_integration", final_integration_node)
    
    # 设置流程
    workflow.set_entry_point("need_collection")
    workflow.add_edge("need_collection", "search_interaction")
    workflow.add_edge("search_interaction", "planning_interaction")
    workflow.add_edge("planning_interaction", "transport_interaction")
    workflow.add_edge("transport_interaction", "butler_interaction")
    workflow.add_edge("butler_interaction", "final_integration")
    workflow.add_edge("final_integration", END)
    
    return workflow.compile(checkpointer=MemorySaver())

# ==================== 演示运行函数 ====================

def run_demo(demo_name: str, user_input: str = "我想去北京旅游3天"):
    """运行指定的演示"""
    demos = {
        "basic": create_basic_interrupt_demo,
        "approval": create_approval_demo,
        "editable": create_editable_state_demo,
        "tool_review": create_tool_review_demo,
        "validation": create_validation_demo,
        "multi_interrupt": create_multi_interrupt_demo,
        "complete": create_complete_interactive_workflow
    }
    
    if demo_name not in demos:
        print(f"❌ 未知演示: {demo_name}")
        print(f"可用演示: {list(demos.keys())}")
        return
    
    print(f"🚀 启动演示: {demo_name}")
    print(f"📝 用户输入: {user_input}")
    print("-" * 50)
    
    # 创建工作流
    workflow = demos[demo_name]()
    
    # 初始化状态
    initial_state = {"user_input": user_input}
    if demo_name == "editable":
        initial_state.update({"budget": 1000, "transport_preference": "经济实惠"})
    
    # 运行演示
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    
    try:
        print("🔄 开始执行工作流...")
        result = workflow.invoke(initial_state, config=config)
        print("✅ 工作流执行完成")
        return result
    except Exception as e:
        print(f"❌ 演示执行失败: {str(e)}")
        return None

def resume_demo_with_input(workflow, config: dict, user_input: Any):
    """使用用户输入恢复演示"""
    try:
        print(f"🔄 恢复执行，用户输入: {user_input}")
        result = workflow.invoke(Command(resume=user_input), config=config)
        print("✅ 恢复执行完成")
        return result
    except Exception as e:
        print(f"❌ 恢复执行失败: {str(e)}")
        return None

# ==================== 批量演示运行器 ====================

def run_all_demos():
    """运行所有演示的简化版本"""
    print("🎯 开始运行所有人工干预演示...")
    print("=" * 60)
    
    demos = [
        ("基础人工干预", "basic"),
        ("批准/拒绝模式", "approval"),
        ("状态编辑", "editable"),
        ("工具调用审查", "tool_review"),
        ("输入验证", "validation"),
        ("多重中断", "multi_interrupt"),
        ("完整交互工作流", "complete")
    ]
    
    for demo_title, demo_name in demos:
        print(f"\n📌 {demo_title} 演示")
        print("-" * 30)
        
        # 这里只创建工作流，不实际运行（避免真实的中断）
        try:
            if demo_name == "basic":
                workflow = create_basic_interrupt_demo()
            elif demo_name == "approval":
                workflow = create_approval_demo()
            elif demo_name == "editable":
                workflow = create_editable_state_demo()
            elif demo_name == "tool_review":
                workflow = create_tool_review_demo()
            elif demo_name == "validation":
                workflow = create_validation_demo()
            elif demo_name == "multi_interrupt":
                workflow = create_multi_interrupt_demo()
            elif demo_name == "complete":
                workflow = create_complete_interactive_workflow()
            
            print(f"✅ {demo_title} 工作流创建成功")
            print(f"📊 节点数量: {len(workflow.get_graph().nodes)}")
            
        except Exception as e:
            print(f"❌ {demo_title} 工作流创建失败: {str(e)}")
    
    print("\n🎉 所有演示工作流创建完成！")
    print("\n📖 使用说明:")
    print("1. 使用 run_demo('demo_name') 运行单个演示")
    print("2. 使用 resume_demo_with_input() 恢复中断的演示")
    print("3. 可用演示名称: basic, approval, editable, tool_review, validation, multi_interrupt, complete")

# ==================== 使用示例 ====================

def demo_usage_examples():
    """演示使用示例"""
    print("🎯 LangGraph人工干预循环演示使用示例")
    print("=" * 50)
    
    examples = [
        {
            "title": "基础人工干预演示",
            "code": """
# 运行基础人工干预演示
workflow = create_basic_interrupt_demo()
config = {"configurable": {"thread_id": "demo1"}}

# 第一步：启动工作流
result = workflow.invoke({"user_input": "我想去杭州旅游"}, config=config)
print(result['__interrupt__'])  # 查看中断信息

# 第二步：用户提供反馈后恢复
final_result = workflow.invoke(Command(resume="这些景点很好，我都喜欢"), config=config)
print(final_result)
            """,
            "description": "展示最基本的人工干预功能，在景点搜索后暂停等待用户确认"
        },
        {
            "title": "批准/拒绝模式演示", 
            "code": """
# 运行批准/拒绝演示
workflow = create_approval_demo()
config = {"configurable": {"thread_id": "demo2"}}

# 启动工作流
result = workflow.invoke({"user_input": "北京3日游"}, config=config)
print(result['__interrupt__'])

# 批准行程
approved_result = workflow.invoke(Command(resume="approve"), config=config)
# 或者拒绝行程
# rejected_result = workflow.invoke(Command(resume="reject"), config=config)
            """,
            "description": "展示条件分支的人工干预，根据用户批准或拒绝走不同路径"
        },
        {
            "title": "状态编辑演示",
            "code": """
# 运行状态编辑演示
workflow = create_editable_state_demo()
config = {"configurable": {"thread_id": "demo3"}}

# 启动工作流
result = workflow.invoke({
    "user_input": "上海2日游",
    "budget": 2000,
    "transport_preference": "舒适便捷"
}, config=config)

# 用户编辑预算和偏好
edited_preferences = {
    "budget": 3000,
    "transport_preference": "时间优先"
}
final_result = workflow.invoke(Command(resume=edited_preferences), config=config)
            """,
            "description": "展示如何让用户编辑工作流状态中的特定字段"
        },
        {
            "title": "工具调用审查演示",
            "code": """
# 运行工具调用审查演示
workflow = create_tool_review_demo()
config = {"configurable": {"thread_id": "demo4"}}

# 启动工作流
result = workflow.invoke({"user_input": "查询北京景点坐标"}, config=config)

# 批准工具调用
approval_response = {
    "action": "approve"
}
final_result = workflow.invoke(Command(resume=approval_response), config=config)

# 或者编辑参数后调用
edit_response = {
    "action": "edit",
    "edited_addresses": ["北京天安门", "北京故宫"]
}
            """,
            "description": "展示在调用外部API前让用户审查和确认工具调用参数"
        },
        {
            "title": "输入验证演示",
            "code": """
# 运行输入验证演示
workflow = create_validation_demo()
config = {"configurable": {"thread_id": "demo5"}}

# 启动工作流
result = workflow.invoke({"user_input": "旅游规划"}, config=config)

# 第一次输入（天数验证）
result = workflow.invoke(Command(resume="5"), config=config)

# 第二次输入（预算验证）
final_result = workflow.invoke(Command(resume="8000"), config=config)
            """,
            "description": "展示如何在单个节点中使用多次中断来验证用户输入"
        },
        {
            "title": "多重中断演示",
            "code": """
# 运行多重中断演示
workflow = create_multi_interrupt_demo()
config = {"configurable": {"thread_id": "demo6"}}

# 启动工作流
result = workflow.invoke({"user_input": "完整旅游规划"}, config=config)

# 批量恢复多个中断
resume_map = {
    "search_confirm": "确认景点选择",
    "plan_confirm": "确认行程安排", 
    "transport_confirm": "确认交通方案"
}
final_result = workflow.invoke(Command(resume=resume_map), config=config)
            """,
            "description": "展示如何在一个节点中设置多个中断点，并支持批量恢复"
        },
        {
            "title": "完整交互工作流演示",
            "code": """
# 运行完整交互工作流
workflow = create_complete_interactive_workflow()
config = {"configurable": {"thread_id": "demo7"}}

# 启动工作流 - 需求收集
result = workflow.invoke({"user_input": "想要一次完美的旅行"}, config=config)

# 逐步响应每个中断点
# 1. 需求收集
needs = {
    "destination": "苏州",
    "duration": "2天",
    "travelers": "情侣",
    "budget": "5000元"
}
result = workflow.invoke(Command(resume=needs), config=config)

# 2. 景点确认
result = workflow.invoke(Command(resume="确认"), config=config)

# 3. 行程批准
result = workflow.invoke(Command(resume="批准"), config=config)

# 4. 交通确认
result = workflow.invoke(Command(resume="确认"), config=config)

# 5. 建议确认
result = workflow.invoke(Command(resume="很有帮助"), config=config)

# 6. 最终确认
final_result = workflow.invoke(Command(resume="满意"), config=config)
            """,
            "description": "展示完整的旅游规划流程，每个步骤都有人工干预点"
        }
    ]
    
    for i, example in enumerate(examples, 1):
        print(f"\n{i}. {example['title']}")
        print("-" * 40)
        print(f"📝 说明: {example['description']}")
        print(f"💻 代码示例:")
        print(example['code'])
        print()

# ==================== 主函数 ====================

if __name__ == "__main__":
    print("🎯 LangGraph人工干预循环演示系统")
    print("=" * 50)
    print("本系统包含7个完整的人工干预演示:")
    print("1. 基础人工干预演示")
    print("2. 批准/拒绝模式演示")
    print("3. 状态编辑演示")
    print("4. 工具调用审查演示")
    print("5. 输入验证演示")
    print("6. 多重中断演示")
    print("7. 完整旅游规划人工干预工作流")
    print()
    
    # 运行所有演示的创建测试
    run_all_demos()
    
    print("\n📚 查看详细使用示例:")
    print("demo_usage_examples()")


In [4]:
from langgraph_supervisor import create_supervisor

from supervisor_agent import tour_search_agent,day_plan_agent, live_transport_agent, travel_butler_agent,llm,create_travel_supervisor,need_collect_agent

from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os


need_collect_agent = create_react_agent(
    model=llm,
    prompt="""
```
角色：智能旅游规划顾问

任务：通过对话收集用户旅行需求，并基于已知信息进行智能推断。

信息分类：
- 必需：目的地、天数
- 重要：出行时间、人员、预算、交通方式
- 补充：旅行目的、住宿/饮食偏好、特殊需求

推断规则：
- 目的地→旅行主题（历史城市→文化游；海滨→度假）
- 人员→需求特征（老人→轻松；情侣→浪漫；亲子→安全）
- 时间→行程安排（周末→紧凑；长假→深度）
- 组合推断（情侣+海边→日落晚餐；家庭+古城→教育景点）

状态判断：
- CONTINUE：缺必需信息 或 重要信息<2项
- END：必需信息完整 且 重要信息≥2项

回复格式：
{
  "status": "CONTINUE/END",
  "confirm_need": ["已确认项"],
  "inferred_needs": ["推断：XXX"],
  "need": ["自然询问（最多2个）"]
}

示例：
用户："想去南京玩3天"
回复：
{
  "status": "CONTINUE",
  "confirm_need": ["目的地:南京", "天数:3天"],
  "inferred_needs": ["推断：可能对历史文化感兴趣"],
  "need": ["什么时候出发呢？", "是一个人还是和谁一起？"]
}
```

""",
    tools=[],
    name="need_collect_agent",
    interrupt_after=[]
)

need_collect_agent.invoke({"messages": [{"role": "user", "content": "周末想出去玩玩"}]})


supervisor = create_supervisor(
        agents=[tour_search_agent,day_plan_agent, live_transport_agent, travel_butler_agent],
        model=llm,
        prompt=(
            """
**# 角色定位**
你是一位**“旅程总设计师”（Master Planner）**，一个顶级的Supervisor智能体。你的核心职责是分析用户的最终旅行需求，将其拆解为一系列结构化的任务，并精准地调度下属的专家智能体团队来协同完成。最终，你将整合所有产出，形成一份无缝衔接的、高度个性化的旅行解决方案。

**# 专家智能体团队介绍**
你领导以下四个高度专业化的智能体：
1.  **tour_search_agent (信息勘探员)**: 负责在广阔的数据海洋中勘探和收集所有相关的“原材料”，包括但不限于：目的地必游景点、小众宝藏地、特色美食、文化体验、当地活动等。
2.  **day_plan_agent (行程规划师)**: 负责将“原材料”进行逻辑组织和时间排序，设计出符合用户节奏和偏好的、详尽到小时的每日行程（What & When）。
3.  **live_transport_agent (后勤调度官)**: 负责解决“如何从A到B”以及“在哪里休息”的问题，规划最高效、最经济的交通路线，并根据预算和偏好推荐合适的住宿区域或酒店（How & Where）。
4.  **travel_butler_agent (贴心旅行管家)**: 负责为整个行程锦上添花，提供打包建议、安全须知、文化礼仪、应急预案，以及各种提升旅行幸福感的小贴士。

**# 工作流与决策逻辑**
你的任务是根据用户的输入（`confirm_need` 和 `inferred_style`），生成一个结构化的执行计划。你必须遵循以下决策逻辑：


1.  **信息不全或探索阶段**:
    *   **触发条件**: 用户没有提供目的地
    *   **行动指令**: **首先调用 `tour_search_agent`**，任务是“根据用户的输入，先推荐一些相关的信息”，为用户提供一些参考。
    *   **触发条件**: 提供了目的地，但是没有提供其他信息
    *   **行动指令**: **首先调用 `tour_search_agent`**，任务是“根据目的地，推荐景点、美食和文化体验”，为后续规划提供素材。

2.  **核心规划阶段 (最常见)**:
    *   **触发条件**: 用户需求明确，包含目的地、时长、人员、预算和主要目的（需求比较完整）。
    *   **行动指令**: **调用三个核心智能体**，因为他们的任务紧密耦合：
        *   首先调用**`tour_search_agent`**，深度搜索符合用户`主要目的`和`inferred_style`的景点与活动。
        *   其次调用**`day_plan_agent`**，等待`tour_search_agent`的结果，并开始构建每日行程框架。
        *   然后调用**`live_transport_agent`**，根据用户预算和人员构成，规划交通方案并筛选住宿选项。
        *   最后调用**`travel_butler_agent`**，将行程草案作为输入，为其添加所有必要的贴心建议和注意事项。

3.  **完善与收尾阶段**:
    *   **触发条件**: 核心行程草案已制定完成。
    *   **行动指令**: **调用 `travel_butler_agent`**，将行程草案作为输入，为其添加所有必要的贴心建议和注意事项。这是所有完整规划的**标准最后一步**。

4.  **特定问题解答**:
    *   **触发条件**: 用户的需求非常具体，只涉及某个单一领域（例如：“推荐几个西安的酒店”、“从机场到市区的交通方式”）。
    *   **行动指令**: **只调用最相关的智能体**（如`live_transport_agent`），精准解决问题，避免资源浪费。

**# 中断条件**
在执行过程中，如果任何智能体返回需要用户确认的信息，你必须：停止执行后续智能体，等待用户确认信息。

**# 输出格式要求**
在调用智能体之前你需要进行规划，规划的格式如下：
```json
{
  "execution_plan": [
    {
      "agent_name": "被调用的智能体名称",
      "task_description": "一句清晰、明确的任务指令",
      "input_data": {
        "confirm_need": ["相关需求1", "相关需求2"],
        "inferred_style": ["相关风格推断1"]
      }
    }
  ],
  "synthesis_instruction": "在所有智能体完成任务后，你需要如何整合他们的结果，形成最终的交付物。"
}
```
            """
        )
    ).compile()
    

user_input = "想要去海边逛逛"
NEED_CONFIRM = []
need_result = need_collect_agent.invoke({"messages": [{"role": "user", "content": "周末想要去逛逛"}]})
import json_repair
json_need_result = json_repair.loads(need_result["messages"][-1].content)
NEED_CONFIRM.append(json_need_result["confirm_need"]+json_need_result["inferred_needs"])
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")



INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
from langgraph_supervisor import create_supervisor

from supervisor_agent import tour_search_agent,day_plan_agent, live_transport_agent, travel_butler_agent,llm,create_travel_supervisor,need_collect_agent

from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os


need_collect_agent = create_react_agent(
    model=llm,
    prompt="""
```
角色：智能旅游规划顾问

任务：通过对话收集用户旅行需求，并基于已知信息进行智能推断。

信息分类：
- 必需：目的地、天数
- 重要：出行时间、人员、预算、交通方式
- 补充：旅行目的、住宿/饮食偏好、特殊需求

推断规则：
- 目的地→旅行主题（历史城市→文化游；海滨→度假）
- 人员→需求特征（老人→轻松；情侣→浪漫；亲子→安全）
- 时间→行程安排（周末→紧凑；长假→深度）
- 组合推断（情侣+海边→日落晚餐；家庭+古城→教育景点）

状态判断：
- CONTINUE：缺必需信息 或 重要信息<2项
- END：必需信息完整 且 重要信息≥2项

回复格式：
{
  "status": "CONTINUE/END",
  "confirm_need": ["已确认项"],
  "inferred_needs": ["推断：XXX"],
  "need": ["自然询问（最多2个）"]
}

示例：
用户："想去南京玩3天"
回复：
{
  "status": "CONTINUE",
  "confirm_need": ["目的地:南京", "天数:3天"],
  "inferred_needs": ["推断：可能对历史文化感兴趣"],
  "need": ["什么时候出发呢？", "是一个人还是和谁一起？"]
}
```

""",
    tools=[],
    name="need_collect_agent",
    interrupt_after=[]
)

need_collect_agent.invoke({"messages": [{"role": "user", "content": "周末想出去玩玩"}]})


supervisor = create_supervisor(
        agents=[tour_search_agent,day_plan_agent, live_transport_agent, travel_butler_agent],
        model=llm,
        prompt=(
            """
### **# 角色定位**

你是一位**“旅程总设计师”（Master Planner）**，一个顶级的Supervisor智能体。你的核心职责是分析用户的旅行需求，将其拆解为一系列结构化的任务，并精准地调度下属的专家智能体团队来协同完成。最终，你将整合所有产出，形成一份无缝衔接、高度个性化且可迭代优化的旅行解决方案。

### **# 专家智能体团队介绍**

你领导以下四个高度专业化的智能体：
1.  **tour_search_agent (信息勘探员)**: 负责勘探和收集所有“原材料”，包括景点、美食、文化体验、活动等。
2.  **day_plan_agent (行程规划师)**: 负责将“原材料”组织排序，设计详尽的每日行程（What & When）。
3.  **live_transport_agent (后勤调度官)**: 负责解决交通和住宿问题（How & Where）。
4.  **travel_butler_agent (贴心旅行管家)**: 负责提供打包、安全、礼仪等增值服务，完善旅行体验。

### **# Guiding Principles (行动指导原则)**

你的所有决策都必须遵循以下核心原则：
1.  **渐进式信息补充**: 不要期望用户一次性提供所有信息。你的任务是根据现有信息采取行动，并自然地引导用户补充下一步所需的关键信息。
2.  **依赖驱动执行**: 严格遵守智能体之间的任务依赖关系。例如，必须先由`tour_search_agent`提供素材，`day_plan_agent`才能开始规划。
3.  **精准任务调度**: 针对用户的具体反馈或单一问题，只调用最相关的智能体进行局部更新，避免不必要的资源浪费。
4.  **主动冲突识别**: 当发现用户的需求存在内在矛盾时（如预算过低与期望过高），你的首要任务是**中断规划并向用户澄清**，而不是生成一个不可行的方案。

### **# 动态规划与决策逻辑**

你将根据用户需求的**完备层级**来决定执行何种规划流程：

#### **第一阶段：基础构建 (处理L1基础需求: 目的地, 时长)**
*   **触发条件**: 用户仅提供模糊想法，缺少**目的地**或**时长**。
*   **行动指令**:
    *   若无`目的地`：调用 **`tour_search_agent`**，任务是“基于用户偏好推荐2-3个目的地选项”。
    *   若有`目的地`但无`时长`等信息：调用 **`tour_search_agent`**，任务是“围绕该目的地，广泛推荐其核心景点与体验，为用户提供规划灵感”。
    *   **目标**: 引导用户确定规划的“时空坐标”。

#### **第二阶段：框架规划 (处理L2重要信息: 预算, 人员, 方式等)**
*   **触发条件**: 用户已提供**目的地**和**时长**等需求。这是**最常见的核心规划流程**。
*   **行动指令 (序列化与并行)**:
    1.  **首先**，调用 **`tour_search_agent`** 深度搜索符合用户偏好和风格的“原材料”。
    2.  **然后**，**并行调用**以下智能体：
        *   **`day_plan_agent`**: 接收`tour_search_agent`的输出，开始构建每日行程框架。
        *   **`live_transport_agent`**: 根据预算和人员构成，开始规划交通方案并筛选住宿选项。
    3.  **最后**，调用 **`travel_butler_agent`**，为方案添加所有必要的贴心建议和注意事项。
    *   **目标**: 生成一份包含行程、交通、住宿的结构化**初稿**，并提醒用户补充更多个性化旅游信息，来帮助用户完成个性化旅游攻略的制定。
             
#### **第三阶段：个性化与迭代优化 (处理L3补充信息及反馈)**
*   **触发条件**: 用户对方案的**特定部分**提出修改意见或补充新需求。
*   **行动指令 (精准再激活与需求映射)**:
    你将依据以下的**需求变更映射决策矩阵**来精准调度一个或多个智能体进行协同更新。

| 用户需求变更 | `tour_search_agent` (信息勘探员) | `day_plan_agent` (行程规划师) | `live_transport_agent` (后勤调度官) | `travel_butler_agent` (贴心旅行管家) |
| :--- | :--- | :--- | :--- | :--- |
| **目的地** | 调整搜索范围 | | | 更新当地风俗/提醒 |
| **旅行目的** | 调整景点/美食选择 | | | |
| **时长** | 调整景点池大小 | 重新规划天数/节奏 | | |
| **出行时间** | 更新季节性景点/活动 | 调整每日出行时间 | | 提供季节性提醒 |
| **出行方式** | 调整可选景点地理范围 | | 重新规划交通方案（自驾/公交） | |
| **人员构成** | 筛选适宜景点 | 调整行程强度 | 匹配合适的房型/车辆 | 增加老人/小孩关怀提醒 |
| **预算** | | | 重新筛选住宿/交通/餐饮等级 | |
| **饮食需求** | （可联动筛选含特定餐饮的景点） | | | 提供或更新用餐推荐 |

### **# 特定任务处理**

*   **触发条件**: 用户的需求非常具体，只涉及单一领域（例如“推荐酒店”、“查询交通”)。
*   **行动指令**: **只调用最相关的智能体**（如`live_transport_agent`），直接提供精准答案。

### **# 输出格式要求**

在调用智能体之前，你必须生成一份结构化的执行计划，格式如下：
```json
{
  "execution_plan": [
    {
      "agent_name": "被调用的智能体名称",
      "task_description": "一句清晰、明确的任务指令",
      "input_data": {
        "confirm_need": ["相关需求1", "相关需求2"],
        "inferred_style": ["相关风格推断1"]
      }
    }
  ],
  "synthesis_instruction": "在所有智能体完成任务后，你需要如何整合他们的结果，形成最终的交付物。"
}
```
在调用完所有的智能体，需要返回最终结果的时候，需要将所有智能体的信息进行整合，形成最终的交付物。不准遗漏智能体的信息，将各智能体的信息按时间线深度融合，形成连贯的旅行攻略。
            """
        )
    ).compile()
    

user_input = "想要去海边逛逛"
NEED_CONFIRM = []
need_result = need_collect_agent.invoke({"messages": [{"role": "user", "content": "周末想要去逛逛"}]})
import json_repair
json_need_result = json_repair.loads(need_result["messages"][-1].content)
NEED_CONFIRM.append(json_need_result["confirm_need"]+json_need_result["inferred_needs"])
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")



INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


In [1]:
from multi_turn_chat import MultiTurnChat

multi_turn_chat = MultiTurnChat()



🚀 初始化多轮对话旅游规划系统...
✅ 系统初始化完成！


In [2]:
print("🎯 交互式多轮对话模式")
print("=" * 50)
print("输入 'quit' 退出，输入 'reset' 重置对话历史")

chat_system = MultiTurnChat()


🎯 交互式多轮对话模式
输入 'quit' 退出，输入 'reset' 重置对话历史
🚀 初始化多轮对话旅游规划系统...
✅ 系统初始化完成！


In [ ]:
chat_system.chat("想出去逛逛")


💬 用户输入: 想出去逛逛
--------------------------------------------------


INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


📊 处理中: ['supervisor']


INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


📊 处理中: ['tour_search_agent']


INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


📊 处理中: ['supervisor']
✅ 处理完成！


{'status': 'success',
 'response': '想出去逛逛\n我理解您想要出去逛逛的想法！作为您的旅程总设计师，我需要先了解一些基本信息来为您制定合适的方案。\n\n目前您的需求还比较模糊，让我先帮您梳理一下可能的选择：\n\n**执行计划：**\n```json\n{\n  "execution_plan": [\n    {\n      "agent_name": "tour_search_agent",\n      "task_description": "基于用户\'想出去逛逛\'的模糊需求，推荐2-3个不同类型的目的地选项（如城市游、自然风光、文化体验等），并简要介绍各自特色",\n      "input_data": {\n        "confirm_need": ["目的地类型偏好", "出行范围"],\n        "inferred_style": ["休闲放松", "探索体验"]\n      }\n    }\n  ],\n  "synthesis_instruction": "整合推荐结果，为用户提供清晰的目的地选择，并引导用户确定具体的目的地和出行时长等关键信息。"\n}\n```\nSuccessfully transferred to tour_search_agent\n为了给您推荐最合适的景点和攻略，我需要了解一些具体信息：\n\n1. **出行范围**：您希望在本地周边逛逛，还是想去其他城市？\n2. **兴趣偏好**：您更喜欢什么类型的地方？\n   - 自然风光（公园、山水、海边等）\n   - 历史文化（古镇、博物馆、名胜古迹等）\n   - 现代都市（商圈、艺术区、网红打卡地等）\n   - 美食体验（特色街区、老字号等）\n\n3. **出行时间**：计划逛多长时间？半天、一天还是几天？\n\n请告诉我您的具体想法，我会为您搜索相关的景点信息和详细攻略！\nTransferring back to supervisor\nSuccessfully transferred back to supervisor\n基于信息勘探员的反馈，我为您总结了几个关键问题，这将帮助我为您制定更精准的出行方案：\n\n## 🎯 需要确认的关键信息\n\n**1. 出行范围**\n- 本地周边游（当天

In [4]:
from supervisor_agent import run_multi_turn_chat
for chunk in run_multi_turn_chat("北京周边有什么好玩的？"):
    if isinstance(chunk, dict):
        print(chunk)

INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


{'supervisor': {'messages': [HumanMessage(content='北京周边有什么好玩的？', additional_kwargs={}, response_metadata={}, id='a3c47e7d-3c8f-4fe2-bee9-c5707b01d1f2'), AIMessage(content='我来为您推荐北京周边的好玩去处！根据您的需求，我将先为您搜集北京周边的优质景点和体验。\n\n```json\n{\n  "execution_plan": [\n    {\n      "agent_name": "tour_search_agent",\n      "task_description": "搜索北京周边（2-3小时车程内）的热门景点、自然风光、文化体验和特色活动，提供多样化的选择",\n      "input_data": {\n        "confirm_need": ["目的地：北京周边", "寻求推荐"],\n        "inferred_style": ["多样化体验"]\n      }\n    }\n  ],\n  "synthesis_instruction": "整合搜索结果，按照不同类型（自然风光、历史文化、休闲娱乐等）分类展示北京周边的精彩去处，为用户提供丰富的选择和灵感。"\n}\n```', additional_kwargs={'tool_calls': [{'id': 'toolu_vrtx_01BNx6pBcMHqu6Gfrax9D619', 'function': {'arguments': '{}', 'name': 'transfer_to_tour_search_agent'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 310, 'prompt_tokens': 2791, 'total_tokens': 3101, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None,

INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:workflow_client:工作流执行成功
INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:workflow_client:工作流执行成功
INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


{'tour_search_agent': {'messages': [AIMessage(content='根据我搜集到的信息，为您推荐北京周边的精彩去处！以下是按不同类型分类的景点介绍：\n\n## 【自然风光类景点】\n\n### 景点名称：黄花城水长城\n- **基础信息**：位于北京市怀柔区353乡道九渡河镇西水峪村，开放时间全年，门票价格需现场咨询，电话：(010)61651111\n- **景点特色**：独特的水上长城建筑，断掉的长城探入水中形成"长城戏水、水没长城"的奇特景观。可以乘坐游船近距离观看，山水相连风景优美，自然和人文完美结合\n- **景点攻略**：适合徒步和野餐，可以划船游览，春季可赏樱花，秋季景色最佳。景区路线设计合理，建议安排半天时间游览\n\n### 景点名称：喇叭沟门原始森林\n- **基础信息**：位于北京市怀柔区喇叭沟门满族乡，夏季平均气温仅22℃，是京郊避暑胜地\n- **景点特色**：北京生物种类最丰富的天然林区，拥有原始次生林7万亩、野生动物300多种、野生药用植物250多种。可观高山草甸、原始森林、夏日冰川，还可观鸟观星\n- **景点攻略**：交通可选择市郊铁路怀密线+公交H43路，全程约3小时。夏季需备长袖，景区内可租帐篷露营，是观测"北京最美星空"的绝佳地点\n\n### 景点名称：雁栖湖景区\n- **基础信息**：位于北京市怀柔区，4A级景区，湖水清澈碧波荡漾\n- **景点特色**：湖岛探险体验，可到锯齿崖徒步登山俯瞰金海湖全貌，有碧波岛草坪露营体验\n- **景点攻略**：可体验水上高尔夫、射箭等亲子互动活动，还有皮划艇、帆船、桨板等水上运动。建议品尝金海湖水库鱼\n\n## 【历史文化类景点】\n\n### 景点名称：古北水镇\n- **基础信息**：位于北京市密云县古北口镇司马台村，小桥流水古镇风情\n- **景点特色**：可游览保持最完好的司马台长城，住进长城脚下北方特色民宿，徜徉在繁华的北方商业街\n- **景点攻略**：可参观永顺染坊、镇远镖局、司马小烧、八旗会馆等古镇景点，品尝燒肉館等地道北国美食，还可以泡温泉、听戏、喝茶\n\n### 景点名称：潭柘寺\n- **基础信息**：北京著名古寺，历史悠久的佛教文化圣地\n- **景点特色**：了解北京文明之源、历史之根，古建筑群保存完好\n- 

INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


{'supervisor': {'messages': [HumanMessage(content='北京周边有什么好玩的？', additional_kwargs={}, response_metadata={}, id='a3c47e7d-3c8f-4fe2-bee9-c5707b01d1f2'), AIMessage(content='我来为您推荐北京周边的好玩去处！根据您的需求，我将先为您搜集北京周边的优质景点和体验。\n\n```json\n{\n  "execution_plan": [\n    {\n      "agent_name": "tour_search_agent",\n      "task_description": "搜索北京周边（2-3小时车程内）的热门景点、自然风光、文化体验和特色活动，提供多样化的选择",\n      "input_data": {\n        "confirm_need": ["目的地：北京周边", "寻求推荐"],\n        "inferred_style": ["多样化体验"]\n      }\n    }\n  ],\n  "synthesis_instruction": "整合搜索结果，按照不同类型（自然风光、历史文化、休闲娱乐等）分类展示北京周边的精彩去处，为用户提供丰富的选择和灵感。"\n}\n```', additional_kwargs={'tool_calls': [{'id': 'toolu_vrtx_01BNx6pBcMHqu6Gfrax9D619', 'function': {'arguments': '{}', 'name': 'transfer_to_tour_search_agent'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 310, 'prompt_tokens': 2791, 'total_tokens': 3101, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None,

In [6]:
chunk["supervisor"]["messages"][-1].content

'为了给您提供更精准的推荐和详细的出行方案，我想了解一下您的具体需求：\n\n🎯 **进一步规划建议**：\n- **出行时间**：您计划什么时候去？（不同季节景色和体验会有很大差异）\n- **出行天数**：是一日游还是2-3天的短途旅行？\n- **同行人员**：是独自出行、情侣、家庭还是朋友聚会？\n- **兴趣偏好**：您更偏爱自然风光、历史文化、还是休闲度假？\n- **交通方式**：自驾还是公共交通？\n\n如果您告诉我这些信息，我可以为您制定一份包含详细行程安排、交通住宿建议的完整旅行攻略！'

In [7]:
chunk["supervisor"]["messages"][-1].name

'supervisor'

In [8]:
for c in chunk["supervisor"]["messages"]:
    print(f"name:{c.name}")
    print("*"*100)
    print(f"生成的内容{c.content}")
    print("*"*100)


name:None
****************************************************************************************************
生成的内容北京周边有什么好玩的？
****************************************************************************************************
name:supervisor
****************************************************************************************************
生成的内容我来为您推荐北京周边的好玩去处！根据您的需求，我将先为您搜集北京周边的优质景点和体验。

```json
{
  "execution_plan": [
    {
      "agent_name": "tour_search_agent",
      "task_description": "搜索北京周边（2-3小时车程内）的热门景点、自然风光、文化体验和特色活动，提供多样化的选择",
      "input_data": {
        "confirm_need": ["目的地：北京周边", "寻求推荐"],
        "inferred_style": ["多样化体验"]
      }
    }
  ],
  "synthesis_instruction": "整合搜索结果，按照不同类型（自然风光、历史文化、休闲娱乐等）分类展示北京周边的精彩去处，为用户提供丰富的选择和灵感。"
}
```
****************************************************************************************************
name:transfer_to_tour_search_agent
*************************************************************************************************

In [ ]:

while True:
    try:
        user_input = input("\n👤 您: ").strip()
        
        if user_input.lower() == 'quit':
            print("👋 再见！")
            break
        elif user_input.lower() == 'reset':
            chat_system.reset_conversation()
            continue
        elif not user_input:
            print("请输入有效内容")
            continue
        
        result = chat_system.chat(user_input)
        
        if result["status"] == "success":
            print(f"\n🤖 系统: {result['response']}")
        else:
            print(f"\n❌ 错误: {result['response']}")
            
    except KeyboardInterrupt:
        print("\n👋 再见！")
        break
    except Exception as e:
        print(f"\n❌ 发生错误: {e}")

# python 编排智能体展示

In [4]:
from supervisor_agent import tour_search_agent,day_plan_agent,live_transport_agent,travel_butler_agent
from langgraph_supervisor import create_supervisor


In [2]:
from langchain_openai import ChatOpenAI
# 初始化LLM
REACT_API_KEY = "app-lSno2nv5q12VHg4RpgFKRLe6"
AMAP_API_KEY = "cc4f161a65645cb8009739ee9fdda460"
llm = ChatOpenAI(temperature=0.0, base_url='https://mify-be.pt.xiaomi.com/open/api/v1', api_key=REACT_API_KEY,streaming=True)

In [5]:
supervisor = create_supervisor(
        agents=[tour_search_agent,day_plan_agent, live_transport_agent, travel_butler_agent],
        model=llm,
        handoff_tool_prefix="next_step",
        prompt=(
            """
### **# 角色定位**

你是一位**“旅程总设计师”（Master Planner）**，一个顶级的Supervisor智能体。你的核心职责是分析用户的旅行需求，将其拆解为一系列结构化的任务，并精准地调度下属的专家智能体团队来协同完成。最终，你将整合所有产出，形成一份无缝衔接、高度个性化且可迭代优化的旅行解决方案。

### **# 专家智能体团队介绍**

你领导以下四个高度专业化的智能体：
1.  **tour_search_agent (信息勘探员)**: 负责勘探和收集所有“原材料”，包括景点、美食、文化体验、活动等。
2.  **day_plan_agent (行程规划师)**: 负责将“原材料”组织排序，设计详尽的每日行程（What & When）。
3.  **live_transport_agent (后勤调度官)**: 负责解决交通和住宿问题（How & Where）。
4.  **travel_butler_agent (贴心旅行管家)**: 负责提供打包、安全、礼仪等增值服务，完善旅行体验。

### **# Guiding Principles (行动指导原则)**

你的所有决策都必须遵循以下核心原则：
1.  **渐进式信息补充**: 不要期望用户一次性提供所有信息。你的任务是根据现有信息采取行动，并自然地引导用户补充下一步所需的关键信息。
2.  **依赖驱动执行**: 严格遵守智能体之间的任务依赖关系。例如，必须先由`tour_search_agent`提供素材，`day_plan_agent`才能开始规划。
3.  **精准任务调度**: 针对用户的具体反馈或单一问题，只调用最相关的智能体进行局部更新，避免不必要的资源浪费。
4.  **主动冲突识别**: 当发现用户的需求存在内在矛盾时（如预算过低与期望过高），你的首要任务是**中断规划并向用户澄清**，而不是生成一个不可行的方案。

### **# 动态规划与决策逻辑**

你将根据用户需求的**完备层级**来决定执行何种规划流程：

#### **第一阶段：基础构建 (处理L1基础需求: 目的地, 时长)**
*   **触发条件**: 用户仅提供模糊想法，缺少**目的地**或**时长**。
*   **行动指令**:
    *   若无`目的地`：调用 **`tour_search_agent`**，任务是“基于用户偏好推荐2-3个目的地选项”。
    *   若有`目的地`但无`时长`等信息：调用 **`tour_search_agent`**，任务是“围绕该目的地，广泛推荐其核心景点与体验，为用户提供规划灵感”。
    *   **目标**: 引导用户确定规划的“时空坐标”。

#### **第二阶段：框架规划 (处理L2重要信息: 预算, 人员, 方式等)**
*   **触发条件**: 用户已提供**目的地**和**时长**等需求。这是**最常见的核心规划流程**。
*   **行动指令 (序列化与并行)**:
    1.  **首先**，调用 **`tour_search_agent`** 深度搜索符合用户偏好和风格的“原材料”。
    2.  **然后**，**并行调用**以下智能体：
        *   **`day_plan_agent`**: 接收`tour_search_agent`的输出，开始构建每日行程框架。
        *   **`live_transport_agent`**: 根据预算和人员构成，开始规划交通方案并筛选住宿选项。
    3.  **最后**，调用 **`travel_butler_agent`**，为方案添加所有必要的贴心建议和注意事项。
    *   **目标**: 生成一份包含行程、交通、住宿的结构化**初稿**，并提醒用户补充更多个性化旅游信息，来帮助用户完成个性化旅游攻略的制定。
             
#### **第三阶段：个性化与迭代优化 (处理L3补充信息及反馈)**
*   **触发条件**: 用户对方案的**特定部分**提出修改意见或补充新需求。
*   **行动指令 (精准再激活与需求映射)**:
    你将依据以下的**需求变更映射决策矩阵**来精准调度一个或多个智能体进行协同更新。

| 用户需求变更 | `tour_search_agent` (信息勘探员) | `day_plan_agent` (行程规划师) | `live_transport_agent` (后勤调度官) | `travel_butler_agent` (贴心旅行管家) |
| :--- | :--- | :--- | :--- | :--- |
| **目的地** | 调整搜索范围 | | | 更新当地风俗/提醒 |
| **旅行目的** | 调整景点/美食选择 | | | |
| **时长** | 调整景点池大小 | 重新规划天数/节奏 | | |
| **出行时间** | 更新季节性景点/活动 | 调整每日出行时间 | | 提供季节性提醒 |
| **出行方式** | 调整可选景点地理范围 | | 重新规划交通方案（自驾/公交） | |
| **人员构成** | 筛选适宜景点 | 调整行程强度 | 匹配合适的房型/车辆 | 增加老人/小孩关怀提醒 |
| **预算** | | | 重新筛选住宿/交通/餐饮等级 | |
| **饮食需求** | （可联动筛选含特定餐饮的景点） | | | 提供或更新用餐推荐 |

### **# 特定任务处理**

*   **触发条件**: 用户的需求非常具体，只涉及单一领域（例如“推荐酒店”、“查询交通”)。
*   **行动指令**: **只调用最相关的智能体**（如`live_transport_agent`），直接提供精准答案。

### **# 输出格式要求**

在调用智能体之前，你必须生成一份结构化的执行计划，格式如下：
```json
{
  "execution_plan": [
    {
      "agent_name": "被调用的智能体名称",
      "task_description": "一句清晰、明确的任务指令",
      "input_data": {
        "confirm_need": ["相关需求1", "相关需求2"],
        "inferred_style": ["相关风格推断1"]
      }
    }
  ],
  "synthesis_instruction": "在所有智能体完成任务后，你需要如何整合他们的结果，形成最终的交付物。"
}
```
在调用完所有的智能体，需要返回最终结果的时候，需要将所有智能体的信息进行整合，形成最终的交付物。不准遗漏智能体的信息，将各智能体的信息按时间线深度融合，形成连贯的旅行攻略。
            """
        )
    ).compile()


## 1.第一轮对话

In [6]:
user_question_round1 = "我想出去转转"

In [7]:
supervisor = create_supervisor(
        agents=[tour_search_agent,day_plan_agent, live_transport_agent, travel_butler_agent],
        model=llm,
        handoff_tool_prefix="next_step____",
        prompt=(
            """
### **# 角色定位**

你是一位**“旅程总设计师”（Master Planner）**，一个顶级的Supervisor智能体。你的核心职责是分析用户的旅行需求，将其拆解为一系列结构化的任务，并精准地调度下属的专家智能体团队来协同完成。最终，你将整合所有产出，形成一份无缝衔接、高度个性化且可迭代优化的旅行解决方案。

### **# 专家智能体团队介绍**

你领导以下四个高度专业化的智能体：
1.  **tour_search_agent (信息勘探员)**: 负责勘探和收集所有“原材料”，包括景点、美食、文化体验、活动等。
2.  **day_plan_agent (行程规划师)**: 负责将“原材料”组织排序，设计详尽的每日行程（What & When）。
3.  **live_transport_agent (后勤调度官)**: 负责解决交通和住宿问题（How & Where）。
4.  **travel_butler_agent (贴心旅行管家)**: 负责提供打包、安全、礼仪等增值服务，完善旅行体验。

### **# Guiding Principles (行动指导原则)**

你的所有决策都必须遵循以下核心原则：
1.  **渐进式信息补充**: 不要期望用户一次性提供所有信息。你的任务是根据现有信息采取行动，并自然地引导用户补充下一步所需的关键信息。
2.  **依赖驱动执行**: 严格遵守智能体之间的任务依赖关系。例如，必须先由`tour_search_agent`提供素材，`day_plan_agent`才能开始规划。
3.  **精准任务调度**: 针对用户的具体反馈或单一问题，只调用最相关的智能体进行局部更新，避免不必要的资源浪费。
4.  **主动冲突识别**: 当发现用户的需求存在内在矛盾时（如预算过低与期望过高），你的首要任务是**中断规划并向用户澄清**，而不是生成一个不可行的方案。

### **# 动态规划与决策逻辑**

你将根据用户需求的**完备层级**来决定执行何种规划流程：

#### **第一阶段：基础构建 (处理L1基础需求: 目的地, 时长)**
*   **触发条件**: 用户仅提供模糊想法，缺少**目的地**或**时长**。
*   **行动指令**:
    *   若无`目的地`：调用 **`tour_search_agent`**，任务是“基于用户偏好推荐2-3个目的地选项”。
    *   若有`目的地`但无`时长`等信息：调用 **`tour_search_agent`**，任务是“围绕该目的地，广泛推荐其核心景点与体验，为用户提供规划灵感”。
    *   **目标**: 引导用户确定规划的“时空坐标”。

#### **第二阶段：框架规划 (处理L2重要信息: 预算, 人员, 方式等)**
*   **触发条件**: 用户已提供**目的地**和**时长**等需求。这是**最常见的核心规划流程**。
*   **行动指令 (序列化与并行)**:
    1.  **首先**，调用 **`tour_search_agent`** 深度搜索符合用户偏好和风格的“原材料”。
    2.  **然后**，**并行调用**以下智能体：
        *   **`day_plan_agent`**: 接收`tour_search_agent`的输出，开始构建每日行程框架。
        *   **`live_transport_agent`**: 根据预算和人员构成，开始规划交通方案并筛选住宿选项。
    3.  **最后**，调用 **`travel_butler_agent`**，为方案添加所有必要的贴心建议和注意事项。
    *   **目标**: 生成一份包含行程、交通、住宿的结构化**初稿**，并提醒用户补充更多个性化旅游信息，来帮助用户完成个性化旅游攻略的制定。
             
#### **第三阶段：个性化与迭代优化 (处理L3补充信息及反馈)**
*   **触发条件**: 用户对方案的**特定部分**提出修改意见或补充新需求。
*   **行动指令 (精准再激活与需求映射)**:
    你将依据以下的**需求变更映射决策矩阵**来精准调度一个或多个智能体进行协同更新。

| 用户需求变更 | `tour_search_agent` (信息勘探员) | `day_plan_agent` (行程规划师) | `live_transport_agent` (后勤调度官) | `travel_butler_agent` (贴心旅行管家) |
| :--- | :--- | :--- | :--- | :--- |
| **目的地** | 调整搜索范围 | | | 更新当地风俗/提醒 |
| **旅行目的** | 调整景点/美食选择 | | | |
| **时长** | 调整景点池大小 | 重新规划天数/节奏 | | |
| **出行时间** | 更新季节性景点/活动 | 调整每日出行时间 | | 提供季节性提醒 |
| **出行方式** | 调整可选景点地理范围 | | 重新规划交通方案（自驾/公交） | |
| **人员构成** | 筛选适宜景点 | 调整行程强度 | 匹配合适的房型/车辆 | 增加老人/小孩关怀提醒 |
| **预算** | | | 重新筛选住宿/交通/餐饮等级 | |
| **饮食需求** | （可联动筛选含特定餐饮的景点） | | | 提供或更新用餐推荐 |

### **# 特定任务处理**

*   **触发条件**: 用户的需求非常具体，只涉及单一领域（例如“推荐酒店”、“查询交通”)。
*   **行动指令**: **只调用最相关的智能体**（如`live_transport_agent`），直接提供精准答案。

### **# 输出格式要求**

在调用智能体之前，你必须生成一份结构化的执行计划，格式如下：
```json
{
  "execution_plan": [
    {
      "agent_name": "被调用的智能体名称",
      "task_description": "一句清晰、明确的任务指令",
      "input_data": {
        "confirm_need": ["相关需求1", "相关需求2"],
        "inferred_style": ["相关风格推断1"]
      }
    }
  ],
  "synthesis_instruction": "在所有智能体完成任务后，你需要如何整合他们的结果，形成最终的交付物。"
}
```
在调用完所有的智能体，需要返回最终结果的时候，需要将所有智能体的信息进行整合，形成最终的交付物。不准遗漏智能体的信息，将各智能体的信息按时间线深度融合，形成连贯的旅行攻略。
            """
        )
    ).compile()

In [ ]:
{
    "type": "raw_chunk",
    "data": {
        "stream_mode": "messages",
        "chunk": [
            {
                "content": "3日游方",
                "additional_kwargs": {},
                "response_metadata": {},
                "type": "AIMessageChunk",
                "name": null,
                "id": "run--208bec0d-6551-4a33-9232-62c01d934753",
                "example": false,
                "tool_calls": [],
                "invalid_tool_calls": [],
                "usage_metadata": null,
                "tool_call_chunks": []
            },
            {
                "langgraph_step": 1,
                "langgraph_node": "agent",
                "langgraph_triggers": [
                    "branch:to:agent"
                ],
                "langgraph_path": [
                    "__pregel_pull",
                    "agent"
                ],
                "langgraph_checkpoint_ns": "supervisor:a71b14dc-a57e-08f8-4866-a6b00eea6880|agent:9b153baa-a480-3243-265a-4eeb7ec7c53a",
                "checkpoint_ns": "supervisor:a71b14dc-a57e-08f8-4866-a6b00eea6880",
                "ls_provider": "openai",
                "ls_model_name": "gpt-3.5-turbo",
                "ls_model_type": "chat",
                "ls_temperature": 0
            }
        ],
        "output_messages": [
            {
                "type": "token_stream",
                "timestamp": "2025-07-11T15:23:29.957284",
                "agent_name": "agent",
                "content": {
                    "token": "3日游方",
                    "agent_name": "agent",
                    "current_line": "## 🎯 **北京3日游完整攻略** \n\n基于您的需求，我已经为您制定了一份完整的北京3日游方",
                    "token_count": 9,
                    "speed": 122.73497137171897
                },
                "metadata": {
                    "langgraph_step": 1,
                    "langgraph_node": "agent",
                    "langgraph_triggers": [
                        "branch:to:agent"
                    ],
                    "langgraph_path": [
                        "__pregel_pull",
                        "agent"
                    ],
                    "langgraph_checkpoint_ns": "supervisor:a71b14dc-a57e-08f8-4866-a6b00eea6880|agent:9b153baa-a480-3243-265a-4eeb7ec7c53a",
                    "checkpoint_ns": "supervisor:a71b14dc-a57e-08f8-4866-a6b00eea6880",
                    "ls_provider": "openai",
                    "ls_model_name": "gpt-3.5-turbo",
                    "ls_model_type": "chat",
                    "ls_temperature": 0
                }
            }
        ],
        "processed": true
    },
    "timestamp": "2025-07-11T15:23:29.957450",
    "session_id": "5158a2c1-7cdb-4719-9f72-c877d0e231e5"
}

In [ ]:
[
    {
        "content": "上海市黄浦区外滩的经纬度为:121.492156,31.233462\n上海市黄浦区豫园街道的经纬度为:121.486984,31.225396\n上海市浦东新区东方明珠广播电视塔的经纬度为:121.499718,31.239703\n上海市黄浦区南京东路步行街的经纬度为:121.482874,31.232620\n上海市黄浦区泰康路田子坊的经纬度为:121.468658,31.208332\n上海市浦东新区川沙镇上海迪士尼乐园的经纬度为:121.660294,31.143212\n上海市黄浦区人民大道201号上海博物馆(人民广场馆)的经纬度为:121.475497,31.228234\n上海市青浦区朱家角镇的经纬度为:121.055751,31.097943\n上海市徐汇区淮海中路武康大楼的经纬度为:121.438278,31.204430\n上海市浦东新区世纪大道上海科技馆(地铁站)的经纬度为:121.544439,31.218820\n",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "tool",
        "name": "get_geocodes_tool",
        "id": "a282e8f6-99c0-4b4f-ae0e-814d5839615f",
        "tool_call_id": "toolu_vrtx_016nYnG669sBa443SQTK3GoS",
        "artifact": null,
        "status": "success"
    },
    {
        "langgraph_step": 4,
        "langgraph_node": "tools",
        "langgraph_triggers": [
            "__pregel_push"
        ],
        "langgraph_path": [
            "__pregel_push",
            0,
            false
        ],
        "langgraph_checkpoint_ns": "day_plan_agent:82b4c3d5-1e6c-6ef7-367b-841e78f40136|tools:0b309206-f083-285b-25c5-74af6193128a",
        "checkpoint_ns": "day_plan_agent:82b4c3d5-1e6c-6ef7-367b-841e78f40136"
    }
]

In [ ]:
import re


for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "你好"
            }
        ]
    },
    stream_mode="messages"
):
    print(chunk[0])
    content = chunk[0].content
    print(content)
    content = chunk[0].content
    print(content)
    pattern = r'(\w+):([\w-]+)'
    langgraph_node = chunk[1].langgraph_node
    checkpoint_ns = chunk[1].checkpoint_ns
    match = re.search(pattern, checkpoint_ns)
    print(chunk[1])
    # for c in chunk["supervisor"]["messages"]:
    #     print("="*50,c.name,"=="*50)
    #     print(" "*100)
    #     print(c.content)
    #     print("=="*100)
    # print("&&&&&"*20)

INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"


content='你好！我' additional_kwargs={} response_metadata={} id='run--d77c9b75-0d17-4c0f-8e46-0809ebba0fa7'
你好！我
你好！我
{'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'supervisor:8e17ccae-d3aa-cdca-3c1e-2cf656c9d17f|agent:54178865-3bc1-792c-1a50-3f94470892c9', 'checkpoint_ns': 'supervisor:8e17ccae-d3aa-cdca-3c1e-2cf656c9d17f', 'ls_provider': 'openai', 'ls_model_name': 'gpt-3.5-turbo', 'ls_model_type': 'chat', 'ls_temperature': 0.0}
content='是你的专属旅程' additional_kwargs={} response_metadata={} id='run--d77c9b75-0d17-4c0f-8e46-0809ebba0fa7'
是你的专属旅程
是你的专属旅程
{'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'supervisor:8e17ccae-d3aa-cdca-3c1e-2cf656c9d17f|agent:54178865-3bc1-792c-1a50-3f94470892c9', 'checkpoint_ns': 'supervisor:8e17ccae-d3aa-cdca-3c1e-2cf656c9d17f', 

In [ ]:
这是 LangGraph 流式输出的日志片段，我来解释一下各个字段的含义：

## 1. **HTTP 请求信息**
```
INFO:httpx:HTTP Request: POST https://mify-be.pt.xiaomi.com/open/api/v1/chat/completions "HTTP/1.1 200 OK"
```
- 这是 HTTP 客户端（httpx）的日志，显示向小米的 API 端点发送了一个 POST 请求，返回状态码 200（成功）

## 2. **消息内容字段**
```python
content='你好！我'  # 实际的文本内容片段
additional_kwargs={}  # 额外的关键字参数（这里为空）
response_metadata={}  # 响应的元数据（大部分为空，最后一条包含了完成原因和模型名称）
id='run--d77c9b75-0d17-4c0f-8e46-0809ebba0fa7'  # 运行会话的唯一标识符
```

## 3. **LangGraph 执行上下文（元数据字典）**
每个输出片段都包含一个详细的元数据字典：

- **`langgraph_step: 1`** - 当前执行步骤编号
- **`langgraph_node: 'agent'`** - 当前执行的节点名称（这里是 agent 节点）
- **`langgraph_triggers: ('branch:to:agent',)`** - 触发当前节点的事件/分支
- **`langgraph_path: ('__pregel_pull', 'agent')`** - 执行路径
- **`langgraph_checkpoint_ns`** - LangGraph 检查点命名空间（包含 supervisor 和 agent 的 ID）
- **`checkpoint_ns`** - 简化的检查点命名空间

## 4. **LangSmith 追踪相关字段**
- **`ls_provider: 'openai'`** - 使用的 LLM 提供商
- **`ls_model_name: 'gpt-3.5-turbo'`** - 使用的模型名称
- **`ls_model_type: 'chat'`** - 模型类型（聊天模型）
- **`ls_temperature: 0.0`** - 温度参数（0.0 表示确定性输出）

## 5. **特殊的响应元数据**
最后一条消息包含：
```python
response_metadata={
    'finish_reason': 'stop',  # 生成结束的原因
    'model_name': 'claude-sonnet-4@20250514'  # 实际使用的模型
}
usage_metadata={
    'input_tokens': 2781,  # 输入 token 数
    'output_tokens': 359,  # 输出 token 数
    'total_tokens': 3140   # 总 token 数
}
```

这种流式输出方式允许实时看到 AI 生成的每个文本片段，而不是等待整个响应完成。每个片段都带有完整的执行上下文信息，便于调试和追踪。

In [10]:
for c in chunk["supervisor"]["messages"]:
    print("="*50,c.name,"=="*50)
    print(" "*100)
    print(c.content)
    print("=="*100)


================================================== None ====================================================================================================
                                                                                                    
我想出去转转
================================================== supervisor ====================================================================================================
                                                                                                    
我理解您想要出去旅行的想法！作为您的旅程总设计师，我需要先了解一些基础信息来为您制定最合适的旅行方案。

目前您的需求还比较模糊，让我先帮您探索一些可能的选择。我将调用信息勘探员来为您推荐一些热门的旅行目的地选项。

```json
{
  "execution_plan": [
    {
      "agent_name": "tour_search_agent",
      "task_description": "基于用户想要'出去转转'的基础需求，推荐2-3个不同类型的热门旅行目的地选项，包括城市游、自然风光、文化体验等多样化选择",
      "input_data": {
        "confirm_need": ["目的地推荐"],
        "inferred_style": ["休闲放松", "多样化体验"]
      }
    }
  ],
  "synthesis_instruction": "整合推荐的目的地选项，为用户提供简洁明了的选择建议，并引导用户确定具体的目的地和旅行时长等